## 1. 라이브러리 불러오기

In [39]:
# 데이터 분석 관련
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

# 데이터 시각화 관련
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid') # matplotlib의 스타일에 관련한 함
## 그래프 출력에 필요한 IPython 명령어
%matplotlib inline 

# Scikit-Learn의 다양한 머신러닝 모듈을 불러옵니다.
# 분류 알고리즘 중에서 선형회귀, 서포트벡터머신, 랜덤포레스트, K-최근접이웃 알고리즘을 사용해보려고 합니다.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

## 2. 데이터 읽기

In [40]:
# 데이터 가져오기
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# 데이터 미리보기
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [41]:
train_df.info()
print('-' * 20)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    

여기서 주의깊게 봐야할 부분은 다음과 같다.

- 각 데이터는 빈 부분이 있는가?
  - 빈 부분이 있다면, drop할 것인가 아니면 default 값으로 채워넣을 것인가
- 데이터는 float64로 변환할 수 있는가
  - 아니면 범주형 데이터로 변환할 수 있는가
  
필요없다고 생각되는 부분은 지우자. 여기서는 PassengerId, Name, Ticket을 지운다.
하지만 이 문제에서 결과물은 'PassengerId', 'Survived' 요소가 필요하므로 훈련 데이터에서만 삭제한다.

In [42]:
train_df = train_df.drop(['PassengerId', 'Name', 'Ticket'], axis = 1)
test_df = test_df.drop(['Name', 'Ticket'], axis = 1)

## 3. 데이터 하나하나 처리하기

### 3.1 Pclass

In [43]:
train_df['Pclass'].value_counts()

Pclass
3    491
1    216
2    184
Name: count, dtype: int64

1, 2, 3등급은 범주형 데이터이므로 더미변수로 지정한다.

In [44]:
pclass_train_dummies = pd.get_dummies(train_df['Pclass'])
pclass_test_dummies = pd.get_dummies(test_df['Pclass'])

# 컬럼명을 문자열로 변경
pclass_train_dummies.columns = ['Pclass_1', 'Pclass_2', 'Pclass_3']
pclass_test_dummies.columns = ['Pclass_1', 'Pclass_2', 'Pclass_3']

train_df.drop(['Pclass'], axis = 1, inplace = True)
test_df.drop(['Pclass'], axis = 1, inplace = True)

train_df = train_df.join(pclass_train_dummies)
test_df = test_df.join(pclass_test_dummies)

### 3.2 Sex

성별도 마찬가지로 또한 one-hot encoding을 진행한다.

In [45]:
sex_train_dummies = pd.get_dummies(train_df['Sex'])
sex_test_dummies = pd.get_dummies(test_df['Sex'])

train_df.drop(['Sex'], axis = 1, inplace = True)
test_df.drop(['Sex'], axis = 1, inplace = True)

train_df = train_df.join(sex_train_dummies)
test_df = test_df.join(sex_test_dummies)

##### 3.3 Age

나이는 연속형 데이터이므로, 큰 처리가 필요없다. (다만 카테고리화를 하여 일부 알고리즘에 더 유용한 결과를 만들 수는 있다고 한다.) 하지만 일부 데이터가 비어있으므로 이를 채울 수 있는 방법에 대해 생각해보자.

1. 랜덤
2. 평균값
3. 중간값
4. 데이터 버리기

우선 이번에는 평균값으로 채우도록 하겠다. 데이터의 통일성을 위해 train 데이터셋의 평균값으로 훈련, 테스트 데이터셋을 채우겠다.

In [46]:
train_df['Age'].fillna(train_df['Age'].mean(), inplace = True)
test_df['Age'].fillna(train_df['Age'].mean(), inplace = True)

/var/folders/bj/ljbr3v1s4vg1cjjtmtss7w3r0000gn/T/ipykernel_38356/3287184556.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Age'].fillna(train_df['Age'].mean(), inplace = True)
/var/folders/bj/ljbr3v1s4vg1cjjtmtss7w3r0000gn/T/ipykernel_38356/3287184556.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setti

### 3.4 SibSp & Parch
형제 자매와 부모님은 가족으로 함께 처리할 수 있지만 바꿀 필요는 없다.

### 3.5 Fare

Fare은 탑승료이다. 특이하게도 test 데이터셋에 1개의 데이터가 비어있다. 아마도 디카프리오인 듯 하다.
데이터 누락이 아닌 무단 탑승이라 생각하고 0을 입력했다.

In [47]:
test_df['Fare'].fillna(0, inplace = True)

/var/folders/bj/ljbr3v1s4vg1cjjtmtss7w3r0000gn/T/ipykernel_38356/1265389997.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['Fare'].fillna(0, inplace = True)


### 3.6 Cabin

Cabin은 객실이다. NaN이 대부분인 데이터이므로 버리도록 하자.

In [48]:
test_df.drop(['Cabin'], axis = 1, inplace = True)
train_df.drop(['Cabin'], axis = 1, inplace = True)


### 3.7 Embarked
Embarked는 탑승 항구를 의미한다. 우선 데이터를 확인해보자.

In [49]:
train_df['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [50]:
test_df['Embarked'].value_counts()

Embarked
S    270
C    102
Q     46
Name: count, dtype: int64

S가 대부분이고 일부 데이터가 비어있는 것을 확인할 수 있다.

In [51]:
train_df['Embarked'].fillna('S', inplace = True)
test_df['Embarked'].fillna('S', inplace = True)

/var/folders/bj/ljbr3v1s4vg1cjjtmtss7w3r0000gn/T/ipykernel_38356/3502955114.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Embarked'].fillna('S', inplace = True)
/var/folders/bj/ljbr3v1s4vg1cjjtmtss7w3r0000gn/T/ipykernel_38356/3502955114.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

In [52]:
embarked_train_dummies = pd.get_dummies(train_df['Embarked'])
embarked_test_dummies = pd.get_dummies(test_df['Embarked'])

embarked_train_dummies.columns = ['S', 'C', 'Q']
embarked_test_dummies.columns = ['S', 'C', 'Q']

train_df.drop(['Embarked'], axis = 1, inplace = True)
test_df.drop(['Embarked'], axis = 1, inplace = True)

train_df = train_df.join(embarked_train_dummies)
test_df = test_df.join(embarked_test_dummies)

## 4. 데이터 나누기
이제 학습용 데이터와 훈련용 데이터를 나누도록 하자.
(정보, 생존 여부)와 같은 형태를 취하여 다음과 같이 데이터를 나눈다.

In [53]:
X_train = train_df.drop(['Survived'], axis = 1)
y_train = train_df['Survived']
X_test = test_df.drop(['PassengerId'], axis = 1).copy()

## 5. 머신러닝 알고리즘 적용하기
이제 로지스틱 회귀, SVC, 랜덤 포레스트, K-최근접 이웃 알고리즘을 각각 적용해서 성능을 비교해보자.

In [54]:
# 로지스틱 회귀 모델 생성
lr_model = LogisticRegression()

# 모델 학습
lr_model.fit(X_train, y_train)

# 예측 및 정확도 계산
lr_score = lr_model.score(X_train, y_train)
print(f'로지스틱 회귀 정확도: {lr_score:.4f}')

# 테스트 데이터 예측
Y_pred = lr_model.predict(X_test)

로지스틱 회귀 정확도: 0.8058


/Users/junghwanryu/.pyenv/versions/3.12.0/envs/kaggle/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [55]:
# SVC 모델 생성
svc_model = SVC()

# 모델 학습
svc_model.fit(X_train, y_train)

# 예측 및 정확도 계산
svc_score = svc_model.score(X_train, y_train)
print(f'SVC 정확도: {svc_score:.4f}')

# 테스트 데이터 예측
Y_pred = svc_model.predict(X_test)

SVC 정확도: 0.6869


In [56]:
# 랜덤 포레스트 모델 생성
rf_model = RandomForestClassifier()

# 모델 학습
rf_model.fit(X_train, y_train)

# 예측 및 정확도 계산
rf_score = rf_model.score(X_train, y_train)
print(f'랜덤 포레스트 정확도: {rf_score:.4f}')

# 테스트 데이터 예측
Y_pred = rf_model.predict(X_test)

랜덤 포레스트 정확도: 0.9820


In [57]:
# KNN 모델 생성
knn_model = KNeighborsClassifier()

# 모델 학습
knn_model.fit(X_train, y_train)

# 예측 및 정확도 계산
knn_score = knn_model.score(X_train, y_train)
print(f'KNN 정확도: {knn_score:.4f}')

# 테스트 데이터 예측
Y_pred = knn_model.predict(X_test)

KNN 정확도: 0.8238


## 6. 제출용 파일 만들기
랜덤 포레스트가 가장 좋은 결과를 내는 것을 알 수 있다. 그 결과로 submission 파일을 만들어서 제출해보자.

In [58]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_score = rf_model.score(X_train, y_train)
Y_pred = rf_model.predict(X_test)

submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': Y_pred
})
submission.to_csv('submission.csv', index = False)